![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Taller: Construcción e implementación de modelos Bagging, Random Forest y XGBoost

En este taller podrán poner en práctica sus conocimientos sobre la construcción e implementación de modelos de Bagging, Random Forest y XGBoost. El taller está constituido por 8 puntos, en los cuales deberan seguir las intrucciones de cada numeral para su desarrollo.

## Datos predicción precio de automóviles

En este taller se usará el conjunto de datos de Car Listings de Kaggle donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

In [12]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
# Importación de librerías
%matplotlib inline
import pandas as pd
import numpy as np

# Lectura de la información de archivo .csv
data = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTrain_carListings.zip')

# Preprocesamiento de datos para el taller
data = data.loc[data['Model'].str.contains('Camry')].drop(['Make', 'State'], axis=1)
data = data.join(pd.get_dummies(data['Model'], prefix='M'))
data = data.drop(['Model'], axis=1)

# Visualización dataset
data.head()

,Price,Year,Mileage,M_Camry,M_Camry4dr,M_CamryBase,M_CamryL,M_CamryLE,M_CamrySE,M_CamryXLE
7,21995,2014,6480,0,0,0,1,0,0,0
11,13995,2014,39972,0,0,0,0,1,0,0
167,17941,2016,18989,0,0,0,0,0,1,0
225,12493,2014,51330,0,0,0,1,0,0,0
270,7994,2007,116065,0,1,0,0,0,0,0


In [14]:
# Separación de variables predictoras (X) y variable de interés (y)
y = data['Price']
X = data.drop(['Price'], axis=1)

In [15]:
# Separación de datos en set de entrenamiento y test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Punto 1 - Árbol de decisión manual

En la celda 1 creen un árbol de decisión **manualmente**  que considere los set de entrenamiento y test definidos anteriormente y presenten el RMSE y MAE del modelo en el set de test.

In [16]:
# Celda 1
def best_split(X, y, num_pct=10):
    features = range(X.shape[1])
    best_split = [0, 0, 0]  # j, split, mse
    # Para todas las varibles 
    for j in features:
        splits = np.percentile(X.iloc[:, j], np.arange(0, 100, 100.0 / (num_pct+1)).tolist())
        splits = np.unique(splits)[1:]
        
        # Para cada partición
        i=0
        for split in splits:
            filter_l = X.iloc[:,j] < split
            y_l = y.loc[filter_l]
            y_r = y.loc[~filter_l]
            mse = np.average(np.square(y_l.mean()-y))
            if i==0:
                best_split = [j, split, mse]
            elif (mse < best_split[2]):
                best_split = [j, split, mse]
            i+=1
    
    return best_split

def tree_grow(X, y, level=0, max_depth=None, num_pct=10):
    
    # Si solo es una observación
    if X.shape[0] == 1:
        tree = dict(y_pred=y.iloc[:1].values[0], y_prob=y.mean(), level=level, split=-1, n_samples=1,mse=0)
        return tree
    
    # Calcular la mejor división
    j, split, mse = best_split(X, y, num_pct)
    
    # Guardar el árbol y estimar la predicción
    y_pred = int(y.mean()) 
    y_prob = (y.sum()+1) / (y.shape[0]+2)  # Corrección Laplace 
    
    tree = dict(y_pred=y_pred, y_prob=y_prob, level=level, split=-1, n_samples=X.shape[0],mse=mse)
    # Revisar el criterio de parada 
    if max_depth is not None:
        if level >= max_depth:
            return tree   
    
    # Continuar creando la partición
    filter_l = X.iloc[:, j] < split
    X_l, y_l = X.loc[filter_l], y.loc[filter_l]
    X_r, y_r = X.loc[~filter_l], y.loc[~filter_l]
    tree['split'] = [j, split]

    # Siguiente iteración para cada partición
    
    tree['sl'] = tree_grow(X_l, y_l, level + 1, max_depth=max_depth, num_pct=num_pct)
    tree['sr'] = tree_grow(X_r, y_r, level + 1, max_depth=max_depth, num_pct=num_pct)
    
    return tree

def tree_predict(X, tree, proba=False):
    
    predicted = np.ones(X.shape[0])

    # Revisar si es el nodo final
    if tree['split'] == -1:
        if not proba:
            predicted = predicted * tree['y_pred']
        else:
            predicted = predicted * tree['y_prob']
            
    else:
        
        j, split = tree['split']
        filter_l = (X.iloc[:, j] < split)
        X_l = X.loc[filter_l]
        X_r = X.loc[~filter_l]

        if X_l.shape[0] == 0:  # Si el nodo izquierdo está vacio solo continua con el derecho 
            predicted[~filter_l] = tree_predict(X_r, tree['sr'], proba)
        elif X_r.shape[0] == 0:  #  Si el nodo derecho está vacio solo continua con el izquierdo
            predicted[filter_l] = tree_predict(X_l, tree['sl'], proba)
        else:
            predicted[filter_l] = tree_predict(X_l, tree['sl'], proba)
            predicted[~filter_l] = tree_predict(X_r, tree['sr'], proba)

    return predicted

In [17]:
tree = tree_grow(X_train, y_train, level=0, max_depth=10, num_pct=10)
tree

{'y_pred': 14508,
 'y_prob': 14503.947817432107,
 'level': 0,
 'split': [8, 1.0],
 'n_samples': 7031,
 'mse': 15508537.05335372,
 'sl': {'y_pred': 14217,
  'y_prob': 14213.416559897501,
  'level': 1,
  'split': [7, 1.0],
  'n_samples': 6242,
  'mse': 16676597.5695051,
  'sl': {'y_pred': 12795,
   'y_prob': 12788.057340720221,
   'level': 2,
   'split': [6, 1.0],
   'n_samples': 3608,
   'mse': 18241215.662586488,
   'sl': {'y_pred': 11201,
    'y_prob': 11190.313827032625,
    'level': 3,
    'split': [5, 1.0],
    'n_samples': 1929,
    'mse': 14085927.385804303,
    'sl': {'y_pred': 10479,
     'y_prob': 10466.337913803873,
     'level': 4,
     'split': [3, 1.0],
     'n_samples': 1599,
     'mse': 15237857.756493198,
     'sl': {'y_pred': 12127,
      'y_prob': 12077.959514170041,
      'level': 5,
      'split': [4, 1.0],
      'n_samples': 492,
      'mse': 22536756.14931653,
      'sl': {'y_pred': 12734,
       'y_prob': 12671.718137254902,
       'level': 6,
       'split': [1,

In [18]:
y_predTreeManual = tree_predict(X_test, tree)
y_predTreeManual

array([13135.,  8622., 17644., ..., 17644., 10882., 11367.])

In [19]:
from sklearn.metrics import mean_squared_error
rmse_TreeManual = mean_squared_error(y_test, y_predTreeManual, squared=False)
rmse_TreeManual

2156.298895352172

In [20]:
from sklearn.metrics import mean_absolute_error as mae
mae_TreeManual = mae(y_test, y_predTreeManual)
mae_TreeManual

1586.6264434180139

### Punto 2 - Bagging manual

En la celda 2 creen un modelo bagging **manualmente** con 10 árboles de clasificación y comenten sobre el desempeño del modelo.

In [21]:
# Celda 2
# Creación de 10 muestras de bootstrap para los 10 árboles
np.random.seed(123)

n_samples = X_train.shape[0]
n_B = 10

samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(1, n_B +1 )]

from sklearn.tree import DecisionTreeRegressor
treereg = DecisionTreeRegressor(max_depth=None, random_state=123)
y_pred = pd.DataFrame(index=y_test.index, columns=[list(range(n_B))])

for i, sample in enumerate(samples):
    X2_train = X_train.iloc[sample,:]
    y2_train = y_train.iloc[sample]
    treereg.fit(X2_train, y2_train)
    y_pred.iloc[:,i] = treereg.predict(X_test)
    
y_pred

,0,1,2,3,4,5,6,7,8,9
257343,14980.0,13993.0,13649.0,13649.0,11788.0,13649.0,13993.0,13990.0,13990.0,13993.0
326011,6492.0,5995.0,5995.0,6987.0,5995.0,5995.0,5995.0,6987.0,5995.0,5995.0
242354,16491.0,16995.0,16491.0,15997.0,15997.0,16491.0,17591.0,16995.0,17404.0,16491.0
266376,21990.0,21990.0,22500.0,21990.0,15900.0,21990.0,22500.0,21990.0,15813.0,21990.0
396954,15988.0,16951.0,15988.0,15988.0,15988.0,17900.0,16951.0,16951.0,15988.0,15988.0
...,...,...,...,...,...,...,...,...,...,...
144298,13836.0,14800.0,14800.0,14800.0,14800.0,14681.0,14800.0,14800.0,13836.0,13836.0
364521,15999.0,14995.0,15999.0,16900.0,15999.0,15999.0,17300.0,15999.0,16900.0,15000.0
120072,23533.0,23533.0,20000.0,17700.0,17700.0,23533.0,17700.0,23533.0,23533.0,20000.0
99878,12989.0,12995.0,12989.0,12995.0,12991.0,12991.0,10995.0,12991.0,12991.0,12893.0


In [22]:
# Desempeño de cada árbol
for i in range(n_B):
    print('Árbol ', i, 'tiene un rmse: ', (mean_squared_error(y_pred.iloc[:,i], y_test, squared=False)))

Árbol  0 tiene un rmse:  2141.613353645869
Árbol  1 tiene un rmse:  2136.3519863123465
Árbol  2 tiene un rmse:  2122.718759132052
Árbol  3 tiene un rmse:  2087.278992468617
Árbol  4 tiene un rmse:  2168.518742842026
Árbol  5 tiene un rmse:  2113.8811455834793
Árbol  6 tiene un rmse:  2127.933470769012
Árbol  7 tiene un rmse:  2184.414847251443
Árbol  8 tiene un rmse:  2138.1071697514985
Árbol  9 tiene un rmse:  2132.097520859104


In [23]:
#Promediamos las predicciones
y_predBaggingManual = y_pred.mean(axis=1)

In [24]:
rmse_BaggingManual = mean_squared_error(y_test, y_predBaggingManual, squared=False)
rmse_BaggingManual

1796.4355868399332

In [25]:
mae_BaggingManual = mae(y_test, y_predBaggingManual)
mae_BaggingManual

1340.0160739030025

**DESEMPEÑO DEL MODELO**

Encontramos que en general en comparación con el modelo de un solo árbol de decisión realizando un ensamblaje por Bagging de 10 árboles mejora notablemente el desempeño de la predicción, donde vemos que la comparación del rmse es de 1796 para el Bagging y 2156 para un modelo con solo un árbol.

Por otro lado el mean absolute error mejor de 1586 de un solo árbol de decisión a 1340 para el modelo de bagging, para este caso se demuestra como este ensamble de 10 árboles mejora en rendimiento a los resultados de un solo árbol.

### Punto 3 - Bagging con librería

En la celda 3, con la librería sklearn, entrenen un modelo bagging con 10 árboles de clasificación y el parámetro `max_features` igual a `log(n_features)` y comenten sobre el desempeño del modelo.

In [ ]:
# Celda 3


### Punto 4 - Random forest con librería

En la celda 4, usando la librería sklearn entrenen un modelo de Randon Forest para clasificación  y comenten sobre el desempeño del modelo.

In [ ]:
# Celda 4


### Punto 5 - Calibración de parámetros Random forest

En la celda 5, calibren los parámetros max_depth, max_features y n_estimators del modelo de Randon Forest para clasificación, comenten sobre el desempeño del modelo y describan cómo cada parámetro afecta el desempeño del modelo.

In [ ]:
# Celda 5


### Punto 6 - XGBoost con librería

En la celda 6 implementen un modelo XGBoost de clasificación con la librería sklearn y comenten sobre el desempeño del modelo.

In [ ]:
# Celda 6


### Punto 7 - Calibración de parámetros XGBoost

En la celda 7 calibren los parámetros learning rate, gamma y colsample_bytree del modelo XGBoost para clasificación, comenten sobre el desempeño del modelo y describan cómo cada parámetro afecta el desempeño del modelo.

In [ ]:
# Celda 7


### Punto 8 - Comparación y análisis de resultados
En la celda 8 comparen los resultados obtenidos de los diferentes modelos (random forest y XGBoost) y comenten las ventajas del mejor modelo y las desventajas del modelo con el menor desempeño.

In [ ]:
# Celda 8
